# Tutorial Part 1 -- Process Pseudobulks + Bulks

In this notebook is Part 1 of a BuDDI analysis. This assumes that you have already QC-ed and preprocessed the data in a format needed for all downstream analyses.

This analysis is the same as the "liver" analysis in the manuscript. In this example we have reference stimulated (female single-cell) data, in this analysis the stimulation is Male:Female maps to CTRL:STIM. Since we want to see how well we predict sex-specific genes in the liver, Male:Female also maps to Train:Test. To run BuDDI when you have no real "STIM" single-cell data, you would run it exactly the same way, but in the code below, you would not loop over the Test/Stim/Female data.

Data format requirements for single-cell data:
- processed data is not scaled
- cells are filtered such that low-quality cells are removed (for example: filter out cells with less than 200 genes and genes expressed in less than 3 cells, and > 5% MT reads)
- data is saved as an AnnData object and you have sample IDs, gene IDs, and cell-type labels
- In the end we will process the data such that we have the following features in the AnnData object that we will use to generate pseudobulks:
  - the observations have columns named: "sample_id", "stim", "isTraining"
  - sample_id: unique IDs for the samples
  - stim: is "STIM" or "CTRL", denotes if the sample is "female" or "male"
  - isTraining: 'Train' or 'Test',  denotes if you can use the sample during training or not. In real use cases, all data will be "Train". To show validation of our experiment, we have access to "Test" data as well.
  


Pseudobulk features:
- we typically generate 1000 pseudobulks per sample with random proportions, for this demo we want it to run faster so we only sample 100
- we generate 100 pseudobulks per sample, per cell type, where the cell-type of interest is >90% of the cell-type, for this demo we want it to run faster so we only sample 10
- we typically sample 5000 cells for each pseudobulk, for this demo we want it to run faster so we only sample 100



# Imports

In [2]:
import sys
sys.path.insert(1, '../../')
from buddi import buddi
from buddi.preprocessing import sc_preprocess


# general imports
import warnings
import numpy as np
import os
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
import collections
import scanpy as sc
import anndata as ad


# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import sklearn as sk

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import pickle
import gzip
from pathlib import Path


2023-09-26 16:11:14.250303: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 16:11:15.171799: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 16:11:20.536422: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/beevol/home/davidson/miniconda3/envs/buddi_analysis/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/beevol/home/davidson/miniconda3/envs/buddi_analysis/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationW

# Parameters

In [5]:
# parameters

aug_data_path = f"{os.getcwd()}/demo_data/augmented_liver_data/"
cibersort_path = f"{os.getcwd()}/demo_data/cibersort_liver/"
data_path = f"{os.getcwd()}/demo_data/processed_sc_liver/"



#####################
### set the study ###
#####################

res_name = "all-liver"
in_name = "liver_droplet_processed"
processed_sc_file = f"{data_path}/{in_name}.h5ad"



# Load and Process data

### Read in data and metadata

In [6]:
# read in the data

adata = sc.read_h5ad(processed_sc_file)

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`



In [7]:
# format metadata


def get_stim_id(in_str):
    out_str = "STIM"
    if in_str == "male":
        out_str = "CTRL"
           
    return(out_str)

# get the columns we need to iterate over for making pseudobulks
adata.obs['scpred_CellType'] = adata.obs['names_merged'].tolist()
adata.obs['sample_id'] = adata.obs['mouse.id'].tolist()
adata.obs['stim'] = [get_stim_id(str(x)) for x in adata.obs['sex'].tolist()]

# make the gene_ids col
adata.var['gene_ids'] = adata.var.index.tolist()

# generate cell-type specific split
adata.obs["isTraining"] = "Train"
stim_idx = np.where(adata.obs.stim == "STIM")[0]
adata.obs["isTraining"][stim_idx] = "Test"

<ipython-input-7-05d5c784bc62>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs["isTraining"][stim_idx] = "Test"


### look at some data stats

In [8]:
# each sample should only have cells in with "STIM" or "CTRL"
tab = adata.obs.groupby(['sample_id', 'stim']).size()
tab.unstack()

stim,CTRL,STIM
sample_id,,
18-F-51,NaN,698.0
30-M-5,362.0,NaN


In [9]:
# see how many cells per cell type
adata.obs["scpred_CellType"].value_counts()


hepatocyte                     296
kupffer                        265
NK                             197
hepatic_sinusoid               107
b_cell                          97
myeloid_leukocyte               74
plasmacytoid_dendritic_cell     17
hepatic_stellate                 7
Name: scpred_CellType, dtype: int64

### write out data to use in CIBERSORTx
we use the signature genes from CIBERSORTx as one of our inputs to BuDDI


In [10]:
# write out data for cibersort
dense_matrix = adata.X.todense()

sc_profile_file = os.path.join(aug_data_path, f"{res_name}_sig.pkl")
sc_profile_path = Path(sc_profile_file)

dense_df = pd.DataFrame(dense_matrix, columns = adata.var['gene_ids'])
dense_df.insert(loc=0, column='scpred_CellType', value=adata.obs["scpred_CellType"].to_list())


pickle.dump( dense_df, open( sc_profile_path, "wb" ) )


# Make pseudobulks

In [19]:
# write out the gene ids
gene_pass = adata.var['gene_ids']
gene_out_file = os.path.join(aug_data_path, f"{res_name}_genes.pkl")
gene_out_path = Path(gene_out_file)
pickle.dump( gene_pass, open( gene_out_path, "wb" ) )

# metadata
sample_order = ['18-F-51', '30-M-5']
stim_order = ['STIM', 'CTRL']
train_order = ['Train', 'Test']

# now generate all the proportions
total_meta_df = pd.DataFrame(columns = ["sample_id", "stim", "isTraining"])

# no cell noise 
len_vector = adata.obs["scpred_CellType"].unique().shape[0]
cell_noise = [np.random.lognormal(0, 0, adata.var['gene_ids'].shape[0]) for i in range(len_vector)]

# cell type order
cell_order = adata.obs.scpred_CellType.unique()

# simulate different number of cells
num_cells = 100
idx = 0
for curr_samp in sample_order:
  for curr_stim in stim_order:
      for curr_train in train_order:

        print(f"running {curr_samp} {curr_stim} {curr_train}")


        # make the pseudobulks
        subset_idx = np.logical_and(adata.obs.sample_id == curr_samp, adata.obs.stim == curr_stim)
        subset_idx = np.where(np.logical_and(subset_idx, adata.obs.isTraining == curr_train))[0]
        if len(subset_idx) == 0:
            continue
        
        temp_adata = adata[subset_idx]

        print("make_prop_and_sum")
        prop_df, pseudobulks_df, test_prop_df, test_pseudobulks_df = sc_preprocess.make_prop_and_sum(temp_adata, 
                                                                                num_samples=1000, 
                                                                                num_cells=num_cells,
                                                                                use_true_prop=False,
                                                                                cell_noise=cell_noise,
                                                                                useSampleNoise=False)
        # number of random pseudobulks
        num_rand_pseudo = pseudobulks_df.shape[0] 

        # get the single cell type proportions
        print("get_single_celltype_prop_matrix")
        ct_prop_df = sc_preprocess.get_single_celltype_prop_matrix(num_samp=100,
                                                                    cell_order=cell_order)

        # now get the cell-type specific pseudobulks
        print("use_prop_make_sum")
        prop_df_sc, pseudobulks_df_sc, _ = sc_preprocess.use_prop_make_sum(temp_adata,  
                                                                            num_cells=num_cells, 
                                                                            props_vec=ct_prop_df, 
                                                                            cell_noise=cell_noise,
                                                                            sample_noise=None,
                                                                            useSampleNoise=False)
        # number of random pseudobulks
        num_ct_pseudo = pseudobulks_df_sc.shape[0] 


        # put them together
        print("concat")        
        prop_df = pd.concat([prop_df,prop_df_sc])
        pseudobulks_df = pd.concat([pseudobulks_df, pseudobulks_df_sc])

        # make the metadata
        num_samps = pseudobulks_df.shape[0] 
        samp_type = ["bulk"]*num_samps
        cell_prop_type = ["random"]*num_rand_pseudo+["cell_type_specific"]*num_ct_pseudo 
        samp_type = ["sc_ref"]*(num_rand_pseudo+num_ct_pseudo)
        
        metadata_df = pd.DataFrame(data = {"sample_id":[curr_samp]*num_samps, 
                                          "stim":[curr_stim]*num_samps,
                                          "isTraining":[curr_train]*num_samps,
                                          "cell_prop_type":cell_prop_type,
                                          "samp_type":samp_type,})

        # make the proportions instead of cell counts
        prop_df = prop_df.div(prop_df.sum(axis=1), axis=0)
        pseudobulk_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_pseudo_splits.pkl")
        prop_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_prop_splits.pkl")
        meta_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_meta_splits.pkl")

        print("write")        
        pseudobulk_path = Path(pseudobulk_file)
        prop_path = Path(prop_file)
        meta_path = Path(meta_file)
        pickle.dump( prop_df, open( prop_path, "wb" ) )
        pickle.dump( pseudobulks_df, open( pseudobulk_path, "wb" ) )
        pickle.dump( metadata_df, open( meta_path, "wb" ) )





running 18-F-51 STIM Train
running 18-F-51 STIM Test
make_prop_and_sum
0
100
get_single_celltype_prop_matrix
use_prop_make_sum
0
concat
write
running 18-F-51 CTRL Train
running 18-F-51 CTRL Test
running 30-M-5 STIM Train
running 30-M-5 STIM Test
running 30-M-5 CTRL Train
make_prop_and_sum
0
100
get_single_celltype_prop_matrix
use_prop_make_sum
0
concat
write
running 30-M-5 CTRL Test


# Process Bulks

### Read in real bulk and format columns

In [22]:
data_path = f"{os.getcwd()}/demo_data/bulk_data/"


in_file = f"{data_path}/GSE132040_190214.csv"
meta_file = f"{data_path}/GSE132040_MACA_Bulk_metadata.csv"
results_file = f"{data_path}/liver_bulk_processed.h5ad"

with open(in_file) as your_data:
    adata = ad.read_csv(your_data, delimiter=',')
    adata = adata.transpose()
    
# add in all the metadata
obs_df = pd.read_csv(meta_file)
obs_df = obs_df.set_index(obs_df["Sample name"] + ".gencode.vM19")

# remake anndata
adata = ad.AnnData(adata.X, obs=obs_df, var=adata.var)

# remove non-gene IDs
gene_idx = np.where(np.logical_not(adata.var_names.str.startswith('__')))[0]
adata = adata[:, gene_idx]

# format the tissue 
adata.obs["tissue"] = [x.split("_")[0] for x in adata.obs["source name"]]

# subset to post-pubescent liver
adata = adata[np.where(adata.obs["tissue"] == "Liver")]
adata = adata[np.where(adata.obs["characteristics: age"] != "1")]



<ipython-input-22-fc0b58648cd9>:25: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["tissue"] = [x.split("_")[0] for x in adata.obs["source name"]]


### format for BuDDI

In [23]:
# Initialize empty column in cell metadata
adata.obs['sample_id'] = adata.obs['source name']

def get_stim_id(in_str):
    out_str = "STIM"
    if in_str == "m":
        out_str = "CTRL"
           
    return(out_str)

adata.obs['stim'] = [get_stim_id(str(x)) for x in adata.obs["characteristics: sex"].tolist()]
adata.var['gene_ids'] = adata.var.index.tolist()

# write it out
del adata.raw
adata.write(results_file)

<ipython-input-23-53c7ec785cb6>:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['sample_id'] = adata.obs['source name']
